# AML Model - VIT hack

### 1) Installing all the dependencies

In [ ]:
!pip install seaborn


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
print("All libraries imported!")

All libraries imported!


### 2) Let's take a look at our dataset

In [ ]:
df=pd.read_csv('Book1.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
df.shape

(50426, 11)

We can see that our dataset contains 50426 rows and 11 columns

let's see the list of all the columns

In [ ]:
columns=df.columns
print(columns)

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


Now let's see how many transactions are flagged as fraud

In [ ]:
print(df['isFraud'].unique())

[0 1]


The ones marked as 0 are not fraudulant whereas the ones markes as 1 are supposed to be fraudulant

Let's count the fraudulant and non-fraudulant labels

In [ ]:
df['isFraud'].value_counts()

0    50326
1      100
Name: isFraud, dtype: int64

We can see, out of 50326 transactions, 100 of them are marked as fraud

Let's now take a look at the type of transactions 

In [ ]:
df.type.unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

We can see that possible type of transactions are -> Payment , Transfer , Cash_out, Debit, Cash_in

Now, let's remove all the unnecessary columns with respect to creating our machine learning model

In [ ]:
df=df.drop(['nameOrig','nameDest', 'isFlaggedFraud','step'],axis =1)
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


Since the type is of string form, we'll one hot encode the contents under the type column

In [ ]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=11d87a3567a9fc78aeccaf2570b04fcb8b62f4d6a817ae030e627c529d5da42f
  Stored in directory: c:\users\shashank verma\appdata\local\pip\cache\wheels\23\9d\42\5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df=pd.get_dummies(df)
df

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.00,160296.36,0.00,0.00,0,0,0,0,1,0
1,1864.28,21249.00,19384.72,0.00,0.00,0,0,0,0,1,0
2,181.00,181.00,0.00,0.00,0.00,1,0,0,0,0,1
3,181.00,181.00,0.00,21182.00,0.00,1,0,1,0,0,0
4,11668.14,41554.00,29885.86,0.00,0.00,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
50421,11141.77,288064.39,276922.63,0.00,0.00,0,0,0,0,1,0
50422,346321.39,276922.63,0.00,5516663.89,6847755.96,0,0,1,0,0,0
50423,334943.53,124.00,0.00,0.00,751505.60,0,0,1,0,0,0
50424,42299.14,2375843.75,2333544.61,0.00,0.00,0,0,0,0,1,0


Now, let's create the training and testing sets

Our Y is supposed to be isFraud column and the remaining portion, our X.

In [ ]:
y=df.isFraud
y.head()

0    0
1    0
2    1
3    1
4    0
Name: isFraud, dtype: int64

In [ ]:
x=df.drop(['isFraud'],axis=1)
x.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,1,0
1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,1,0
2,181.00,181.0,0.00,0.0,0.0,0,0,0,0,1
3,181.00,181.0,0.00,21182.0,0.0,0,1,0,0,0
4,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,1,0


In [ ]:
print("Shape of X = {}".format(x.shape))
print("Shape of Y = {}".format(y.shape))

Shape of X = (50426, 10)
Shape of Y = (50426,)


Now we'll normalize our data set

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
x=df.values[:,1:]
x

array([[1.70136000e+05, 1.60296360e+05, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.12490000e+04, 1.93847200e+04, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.81000000e+02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.24000000e+02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.37584375e+06, 2.33354461e+06, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.33354461e+06, 2.32028150e+06, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [ ]:
x=np.nan_to_num(x)
cluster_dataset=StandardScaler().fit_transform(x)
cluster_dataset

array([[-0.25524801, -0.26165136, -0.3470727 , ..., -0.11355928,
         1.13409332, -0.32150471],
       [-0.32309633, -0.324786  , -0.3470727 , ..., -0.11355928,
         1.13409332, -0.32150471],
       [-0.33269709, -0.33347121, -0.3470727 , ..., -0.11355928,
        -0.88176165,  3.11037433],
       ...,
       [-0.33272306, -0.33347121, -0.3470727 , ..., -0.11355928,
        -0.88176165, -0.32150471],
       [ 0.74990057,  0.71205991, -0.3470727 , ..., -0.11355928,
         1.13409332, -0.32150471],
       [ 0.73062471,  0.70611745, -0.3470727 , ..., -0.11355928,
         1.13409332, -0.32150471]])

we need to divide all the transactions into two clusters, first one being the normal transaction and the second one being the flagged one.


In [ ]:
num=2
k_means=KMeans(init='k-means++',n_clusters=num,n_init=2)
k_means.fit(cluster_dataset)
labels=k_means.labels_
labels


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
labels.shape

(50426,)

In [ ]:
count=0  #total 100 labels were marked as fraud i.e 1
diffcount=0 #wrongly classified
z=50426
for i in range (z):
    if labels[i]==1:
        count+=1
    elif labels[i]==0:
        diffcount+=1
        
print("Fraud labelled transaction = {}".format(count))
print("normal transactions = {}".format(diffcount))

Fraud labelled transaction = 9169
normal transactions = 41257
